# 模型部署：ONNX导出、Flask API部署、Docker打包
如何将训练好的模型导出为ONNX格式，并使用Flask API进行部署，最后使用Docker打包，提供可访问的HTTP预测接口。

In [ ]:
# 导入必要的库
import torch
import onnx
from flask import Flask, request, jsonify
import numpy as np


## 模型导出为ONNX格式(开放式神经网络交换)

In [ ]:
# 假设我们有一个训练好的PyTorch模型
class SimpleModel(torch.nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = torch.nn.Linear(10, 1)
    def forward(self, x):
        return self.fc(x)

model = SimpleModel()
dummy_input = torch.randn(1, 10)
torch.onnx.export(model, dummy_input, "simple_model.onnx")
print("模型已导出为ONNX格式")

## 使用Flask API进行部署

In [ ]:
# 创建Flask应用
app = Flask(__name__)

# 加载ONNX模型
onnx_model = onnx.load("simple_model.onnx")
session = onnxruntime.InferenceSession(onnx_model.SerializeToString())

# 定义预测接口
@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    input_data = np.array(data['input']).astype(np.float32)
    input_name = session.get_inputs()[0].name
    result = session.run(None, {input_name: input_data})
    return jsonify({'prediction': result[0].tolist()})

# 运行Flask应用
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

## 使用Docker打包

In [ ]:
# 创建Dockerfile
dockerfile_content = '''
FROM python:3.8-slim
WORKDIR /app
COPY . /app
RUN pip install flask onnx onnxruntime
EXPOSE 5000
CMD ["python", "app.py"]
'''
with open('Dockerfile', 'w') as f:
    f.write(dockerfile_content)
print("Dockerfile已创建")

## 构建和运行Docker镜像

In [ ]:
# 构建Docker镜像
!docker build -t simple_model_api .

# 运行Docker容器
!docker run -d -p 5000:5000 simple_model_api
print("Docker容器已启动，API在http://localhost:5000/predict")